# Actuarial Standards of Practice (ASOP) Q&A Machine using Retrieval Augmented Generation (RAG)
This project aims to create a Retrieval-Augmented Generation (RAG) process for actuaries to ask questions on a set of Actuarial Standards of Practice (ASOP) documents. The RAG process utilizes the power of the Large Language Model (LLM) to provide answers to questions on ASOPs.

However, RAG is not without challenges, i.e., hallucination and inaccuracy. This code allows verifiability by providing the context it used to arrive at those answers. This process enables actuaries to validate the information provided by the LLM, empowering them to make informed decisions. By combining the capabilities of LLM with verifiability, this code offers actuaries a robust tool to leverage LLM technology effectively and extract maximum value.

The current example uses either OpenAI's GPT 3.5 turbo or a local LLM. Using local LLM can address potential data privacy or security concerns.

# 1. Initial Setup
This setup includes loading environment variables from a `.env` file, setting the required environment variables, and importing the necessary modules for further processing. It ensures that the code has access to the required APIs and functions for the subsequent tasks.


In [2]:
# Initial set up
from dotenv import load_dotenv
import os

# Load the variables from .env file and set the API key (or user may manually set the API key)
load_dotenv()  # This loads the variables from .env (not part of repo)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["LANGCHAIN_TRACING_V2"] = "true" # use when you want to debug or monitor the performance of your langchain applications
#os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY') # use when accessing cloud-based language models or services that langchain integrates with

# Import the necessary modules
import bs4
from langchain import hub
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel # for RAG with source
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from IPython.display import display, Markdown, Latex
import glob
import chromadb

In [3]:
# Change to False if using local models instead of OpenAI models
use_OpenAI = False

if use_OpenAI:
    embeddings_model = OpenAIEmbeddings()
    db_directory = "../data/chroma_db1"
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", 
                     temperature=0) # context window size 16k for GPT 3.5 Turbo

else: #Open source models used here are for illustration and educational purposes
    embeddings_model = GPT4AllEmbeddings()
    db_directory = "../data/chroma_db2"
    # Create a local large language model for augmented generation
    # Ollama is one way to easily run inference (especially on macOS)
    llm = Ollama(model="solar:10.7b-instruct-v1-q5_K_M")

100%|█████████████████████████████████████| 45.9M/45.9M [00:00<00:00, 71.2MiB/s]


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


# 2. Load PDF Files and Convert to a Vector DB
1. Create a function to load and extract text from PDF files in a specified folder. It defines a function called `load_pdfs_from_folder()` that takes a folder path as input and returns a list of extracted text documents from the PDF files in that folder.

2. In the example, the folder path `../data/ASOP` is used, but you can modify it to point to your desired folder.

3. By calling the `load_pdfs_from_folder()` function with the folder path, the code loads the PDF files, extracts the text using the PyPDFLoader, and stores the extracted text documents in the `docs` list.

4. After loading and extracting the text, a `RecursiveCharacterTextSplitter` object is created with specific parameters for chunking the documents. The `split_documents()` method is then used to split the documents into smaller chunks based on the specified parameters.

5. Finally, a Chroma vectorstore is created from the document splits. The vectorstore uses the defined embedding model for embedding the chunks and is saved to the predefined directory.

In [5]:
# Run only when the DB directory is empty
if not os.path.exists(db_directory) or not os.listdir(db_directory):
    # Define a function to load and extract text from PDFs in a folder
    def load_pdfs_from_folder(folder_path):
        # Get a list of PDF files in the specified folder
        pdf_files = glob.glob(f"{folder_path}/*.pdf")
        docs = []
        for pdf_file in pdf_files:
            # Load the PDF file using the PyPDFLoader
            loader = PyPDFLoader(pdf_file) 
            # Extract the text from the PDF and add it to the docs list
            docs.extend(loader.load())
        return docs
    
    # Example folder path
    folder_path = '../data/ASOP'
    
    # Call the function to load and extract text from PDFs in the specified folder
    docs = load_pdfs_from_folder(folder_path)
    
    # Create a text splitter object with specified parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=200,
        length_function=len,)
    
    # Split the documents into chunks using the text splitter
    splits = text_splitter.split_documents(docs)
    
    # Create a Chroma vector database from the document splits, using OpenAIEmbeddings for embedding
    vectorstore = Chroma.from_documents(documents=splits, 
                                        embedding=embeddings_model, 
                                        persist_directory=db_directory)

# 3. Retrieve from the Vector DB 

In [6]:
# Get a Chroma vector database with specified parameters
vectorstore = Chroma(embedding_function=embeddings_model, 
                     persist_directory=db_directory)

In [7]:
## Retrieve and RAG chain

# Create a retriever using the vector database as the search source
retriever = vectorstore.as_retriever(search_type="mmr", 
                                     search_kwargs={'k': 6, 'lambda_mult': 0.25}) 
# Use MMR (Maximum Marginal Relevance) to find a set of documents that are both similar to the input query and diverse among themselves
# Increase the number of documents to get, and increase diversity (lambda mult 0.5 being default, 0 being the most diverse, 1 being the least)

# Load the RAG (Retrieval-Augmented Generation) prompt
prompt = hub.pull("rlm/rag-prompt")

# Define a function to format the documents with their sources and pages
def format_docs_with_sources(docs):
    formatted_docs = "\n\n".join(doc.page_content for doc in docs)
    sources_pages = "\n".join(f"{doc.metadata['source']} (Page {doc.metadata['page'] + 1})" for doc in docs)
    # Added 1 to the page number assuming 'page' starts at 0 and we want to present it in a user-friendly way

    return f"Documents:\n{formatted_docs}\n\nSources and Pages:\n{sources_pages}"

# Create a RAG chain using the formatted documents as the context
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs_with_sources(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# Create a parallel chain for retrieving and generating answers
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

# 4. Generate Q&A

In [8]:
def generate_output():
    # Prompt the user for a question on ASOP
    usr_input = input("What is your question on ASOP?: ")

    # Invoke the RAG chain with the user input as the question
    output = rag_chain_with_source.invoke(usr_input)

    # Generate the Markdown output with the question, answer, and context
    markdown_output = "### Question\n{}\n\n### Answer\n{}\n\n### Context\n".format(output['question'], output['answer'])

    last_page_content = None  # Variable to store the last page content
    i = 1 # Source indicator

    # Iterate over the context documents to format and include them in the output
    for doc in output['context']:
        current_page_content = doc.page_content.replace('\n', '  \n')  # Get the current page content
        
        # Check if the current content is different from the last one
        if current_page_content != last_page_content:
            markdown_output += "- **Source {}**: {}, page {}:\n\n{}\n".format(i, doc.metadata['source'], doc.metadata['page'], current_page_content)
            i = i + 1
        last_page_content = current_page_content  # Update the last page content
    
    # Display the Markdown output
    display(Markdown(markdown_output))

### Example questions related to ASOPs
- explain ASOP No. 14
- How are expenses relfected in cash flow testing based on ASOP No. 22?
- What is catastrophe risk?
- When do I update assumptions?
- What should I do when I do not have credible data to develop non-economic assumptions?

In [10]:
generate_output()

What is your question on ASOP?:  How are expenses relfected in cash flow testing based on ASOP No. 22?


### Question
How are expenses relfected in cash flow testing based on ASOP No. 22?

### Answer
Expenses are reflected in cash flow testing based on ASOP No. 22 through the consideration of administrative or investment fees and other payments borne by the plan, as mentioned in ASOP No. 4's definition of "expenses." The actuary should also adjust the investment return assumption to account for expenses that might not be otherwise recognized when performing cash flow testing on health benefit plans or pension obligations.

### Context
- **Source 1**: ../data/ASOP/asop022_167.pdf, page 17:

July 1991). In addition, in July 1990, the ASB adopted ASOP No. 14, When to Do Cash Flow   
Testing for Life and Health Insurance Companies,  to provide guidance in determining whether   
or not to do cash flow testing in form ing a professional opinion or recommendation.
- **Source 2**: ../data/ASOP/asop007_128.pdf, page 22:

draft of ASOP No. 22. A commentator on ASOP No. 22 thought that the definition should include surplus   
notes.   The task force agreed and added a definition of “o ther liability cash flows,” which includes a reference   
to surplus notes, to both ASOP No. 7 and No. 22.    
Section 2.15, Policy Cash Flows (previously section 2.14)   
Comment   Response One commentator noted that the definition did not treat  premium taxes properly, as premium taxes are not   
paid on behalf of policyholders, but rather are paid as required by law.   The task force agreed with this comment and changed the definition accordingly.    
SECTION 3.  ANALYSIS OF ISSUES AND RECOMMENDED PRACTICES   
Section 3.2.1, Reasons for Cash Flow Testing, and 3.2.2, Cash Flow Testing is Not Always Necessary   
Comment   
  Response A few commentators questioned the use of the phrases  “long duration” and “short-term,” and noted that
- **Source 3**: ../data/ASOP/asop004_205.pdf, page 11:

ASOP No. 4—Doc. No. 205   
 4   
2.9 Cost Allocation Procedure A procedure that determines the periodic cost  for a plan (for   
example, the procedure to determi ne the net periodic pension co st under accounting   
standards). The procedure uses an actuarial cost method , and may use an asset valuation   
method or an amortization method .    
 2.10 Expenses—Administrative or investment fees or other paymen ts borne or expected to be   
borne by the plan.    
 2.11  
 Funded Status—Any comparison of a particular measure of plan as sets to a particular   
measure of pension obligations.   
 2.12 Funding Valuation—A measuremen t of pension obligations or projection of cash flows   
performed by the actuary intended to be used by the principal t o determine plan   
contributions or to evaluate the adequacy of specified contribu tion levels to support benefit   
provisions.    
 2.13 Gain and Loss Analysis  
—An analysis of the effect on the plan’s funded status  between
- **Source 4**: ../data/ASOP/asop022_167.pdf, page 9:

ASOP No. 22—September 2001    
   
 32.8 Cash Flow Testing ⎯A form of cash flow analysis i nvolving the projection and comparison   
of the timing and amount of cash flows resulting from economic and other assumptions.    
   
2.9 Gross Premium Reserve ⎯The actuarial present value of benefits, expenses, and related   
amounts less the actuarial present value of premiums and related amounts.   
 2.10 Gross Premium Reserve Test  
—The comparison of the gross premium reserve computed   
under one or more scenarios to the financial statement reserve.   
 2.11 Health Benefit Plan  
—A contract or other financia l arrangement providing medical,   
prescription drug, dental, vision, disability in come, accidental death and dismemberment,   
long-term care, or other health-related benefits, whether on a reimbursement, indemnity, or service benefit basis, regardless of the form  of the risk-assuming entity, including health   
benefit plans provided by self-insured or government plan sponsors.    
 2.12 Insurer
- **Source 5**: ../data/ASOP/asop022_167.pdf, page 20:

possible.   The task force agreed that the intention was not to exclude other methods from being considered and   
clarified the language.    
Comment    Response One commentator noted that the wording in the first paragraph did not properly discuss the two separate   
issues of testing existing (in force) cash flows vs. testing combined cash flows. On the latter, it was   
noted that in some situations changes in asset and liability cash flows may offset.   The task force agreed and expanded upon the expos ure draft wording to make this clearer.
- **Source 6**: ../data/ASOP/asop027_197.pdf, page 14:

expenses may be paid from plan assets. To the extent such expenses are not   
otherwise recognized, the actuary should reduce the investment return   
assumption to reflect these expenses.


# 5. References
- https://www.actuarialstandardsboard.org/standards-of-practice/
- https://python.langchain.com/docs/use_cases/question_answering/quickstart
- https://python.langchain.com/docs/use_cases/question_answering/sources
- https://chat.langchain.com/